In [1]:
%pip install --upgrade transformers langchain langchain-community

  Using cached dataclasses_json-0.6.4-py3-none-any.whl.metadata (25 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached marshmallow-3.20.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached jsonpointer-2.4-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
   ---------------------------------------- 0.0/816.1 kB ? eta -:--:--
   - -------------------------------------- 30.7/816.1 kB ? eta -:--:--
   ----- ---------------------------------- 112.6/816.1 kB 2.2 MB/s eta 0:00:01
   ------------------------- -------------- 512.0/816.1 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 816.1/816.1 kB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------- ----------------------- 0.7/1.7 MB 22.1 MB/s eta 0:00:01
 

In [4]:
%pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [16]:
# from langchain.llms import TextGenerationWrapper
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import TextLoader
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.vectorstores import FAISS
from langchain.llms.huggingface_hub import HuggingFaceHub
from langchain_community.chat_models.huggingface import ChatHuggingFace
from dotenv import load_dotenv, get_key
import os
load_dotenv()


True

In [18]:
# Load Hugging Face model mixtrel 8x7b from huggingface hub
os.environ["HUGGINGFACEHUB_API_TOKEN"] = get_key(key_to_get="HUGGINGFACEHUB_API_KEY",dotenv_path=".env")
llm = HuggingFaceHub(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 1000,
        "top_k": 30,
        "temperature": 0.5,
        "repetition_penalty": 1.03,
    },
)
model = ChatHuggingFace(llm=llm)
# Load conversation history (replace with your own text loader)
loader = TextLoader("conversation_history.txt")
documents = loader.load()

c:\Users\ditec\anaconda3\envs\crescendo\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_text_gen_inference.HuggingFaceTextGenInference` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.


In [60]:
from langchain_community.llms import HuggingFaceHub
llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

In [97]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(template= "You're a helpful data assistant which can answer questions on following multiple reviews of a perticular product {reviews}",input_variables=["reviews"])

user_template= PromptTemplate(template="{user_input}", input_variables=["user_input"])

reviews = ''' 
Great Product , Value for Money, Original product, great quality. Good quality Very very nice Toothpaste Le sakte ho Worth the value I bought it for @281 pack of 4×200g and 2nd time @546 for 8×200gToothpaste is no doubt best but the packaging is ordenary it should be improved and sometime the price gets high otherwise happy with the product 😊😊 This is very good paste. Good 👍 Product quality is very good at good price of 277 1) Effect on teeth: It claims to solve many teeth problems, maybe there is effect but I didnt notice any.2) Sensitivity: Maybe not great for sensitive teeth. But not too bad.3) Taste: Tastes like some ayurvedic recipe. Not bad.4) Ingredients: Has many natural ingredients and minimal chemicals compared to other pastes.5) Price: The 3+1 combos are so much cheaper than the single ones. There are 100g, 150g, 200g variants in 3+1 combos, so check whichever you can afford and whichever is on biggger discount.Overall: Holygrail toothpaste. Would repurchase till I find a better cheaper alternative. Nice Indian product world's very reasonablly with very reasonable rate Good
'''

final_prompt = prompt.format(reviews=reviews)


print(final_prompt)

chat_model = ChatHuggingFace(llm=llm)


You're a helpful data assistant which can answer questions on following multiple reviews of a perticular product  
Great Product , Value for Money, Original product, great quality. Good quality Very very nice Toothpaste Le sakte ho Worth the value I bought it for @281 pack of 4×200g and 2nd time @546 for 8×200gToothpaste is no doubt best but the packaging is ordenary it should be improved and sometime the price gets high otherwise happy with the product 😊😊 This is very good paste. Good 👍 Product quality is very good at good price of 277 1) Effect on teeth: It claims to solve many teeth problems, maybe there is effect but I didnt notice any.2) Sensitivity: Maybe not great for sensitive teeth. But not too bad.3) Taste: Tastes like some ayurvedic recipe. Not bad.4) Ingredients: Has many natural ingredients and minimal chemicals compared to other pastes.5) Price: The 3+1 combos are so much cheaper than the single ones. There are 100g, 150g, 200g variants in 3+1 combos, so check whichever y

In [112]:
def chatwithbot(txt:str):
    messages = [
    SystemMessage(content=final_prompt),
    HumanMessage(content=user_template.format(user_input=txt))
    ]
    res = chat_model(messages).content
    # trim res to remove the prompt to keep only the response after word assistant
    res = res[res.find("assistant")+9:]
    return res
    # remove the prompt from the response and return the last response


    

In [113]:
print(chatwithbot("Is it worth buying?"))

 which can answer questions on following multiple reviews of a perticular product  
Great Product , Value for Money, Original product, great quality. Good quality Very very nice Toothpaste Le sakte ho Worth the value I bought it for @281 pack of 4×200g and 2nd time @546 for 8×200gToothpaste is no doubt best but the packaging is ordenary it should be improved and sometime the price gets high otherwise happy with the product 😊😊 This is very good paste. Good 👍 Product quality is very good at good price of 277 1) Effect on teeth: It claims to solve many teeth problems, maybe there is effect but I didnt notice any.2) Sensitivity: Maybe not great for sensitive teeth. But not too bad.3) Taste: Tastes like some ayurvedic recipe. Not bad.4) Ingredients: Has many natural ingredients and minimal chemicals compared to other pastes.5) Price: The 3+1 combos are so much cheaper than the single ones. There are 100g, 150g, 200g variants in 3+1 combos, so check whichever you can afford and whichever is 

UnboundLocalError: local variable 'messages' referenced before assignment